----
#Colab Work

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/Senior_Project/Snake_Plus_Plus/Snake_Walls/')
!pwd

/content
/content/drive/MyDrive/Colab Notebooks/Senior_Project/Snake_Plus_Plus/Snake_Walls


In [ ]:
!pip install keras-rl2==1.0.4
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


-------------

# Imports

In [ ]:

from PIL import Image  # To transform the image in the Processor
import numpy as np
import gym

# Convolutional Backbone Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Convolution2D, Permute
from tensorflow.keras.optimizers import Adam

# Keras-RL
from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
from tensorflow.keras.models import load_model


In [ ]:
!pip install -e walls

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Colab%20Notebooks/Senior_Project/Snake_Plus_Plus/Snake_Walls/walls
  Preparing metadata (setup.py) ... done
  Attempting uninstall: gymwalls
    Found existing installation: gymwalls 0.0.1
    Can't uninstall 'gymwalls'. No files were found to uninstall.
  Running setup.py develop for gymwalls


In [ ]:
os.environ['SDL_VIDEODRIVER']='dummy'

In [ ]:
env = gym.make("walls:walls_v0")
nb_actions = env.action_space.n


/usr/local/lib/python3.8/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


-----
# Image Processing

In [ ]:
nb_actions

4

In [ ]:
IMG_SHAPE = (84, 84)
WINDOW_LENGTH = 4


In [ ]:
class ImageProcessor(Processor):
    def process_observation(self, observation):
        # First convert the numpy array to a PIL Image
        img = Image.fromarray(observation)
        # Then resize the image
        img = img.resize(IMG_SHAPE)
        # And convert it to grayscale  (The L stands for luminance)
        img = img.convert("L")
        # Convert the image back to a numpy array and finally return the image
        img = np.array(img)
        return img.astype('uint8')  # saves storage in experience memory
    
    def process_state_batch(self, batch):

        # We divide the observations by 255 to compress it into the intervall [0, 1].
        # This supports the training of the network
        # We perform this operation here to save memory.
        processed_batch = batch.astype('float32') / 255.
        return processed_batch

    def process_reward(self, reward):
        return np.clip(reward, -1., 1.)


## Model Creation

**NOTE: Depending on your custom environment, this model will vary greatly, try reading papers that are solving similar problems to your own!**

In [ ]:
input_shape = (WINDOW_LENGTH, IMG_SHAPE[0], IMG_SHAPE[1])
input_shape

(4, 84, 84)

In [ ]:
# MODEL LOADING

#model = load_model("test_dqn_snake_weights_100000.h5f")

# MODEL FROM SCRATCH
model = Sequential()
model.add(Permute((2, 3, 1), input_shape=input_shape))

model.add(Convolution2D(32, (8, 8), strides=(4, 4),kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (4, 4), strides=(2, 2), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Convolution2D(64, (3, 3), strides=(1, 1), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

#print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 permute (Permute)           (None, 84, 84, 4)         0         
                                                                 
 conv2d (Conv2D)             (None, 20, 20, 32)        8224      
                                                                 
 activation (Activation)     (None, 20, 20, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 64)          32832     
                                                                 
 activation_1 (Activation)   (None, 9, 9, 64)          0         
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          36928     
                                                                 
 activation_2 (Activation)   (None, 7, 7, 64)          0

----
## Creating the Agent

In [ ]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)


In [ ]:
processor = ImageProcessor()


In [ ]:
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05,
                              nb_steps=1000000)


In [ ]:
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000,
              train_interval=4, delta_clip=1)

In [ ]:
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])


In [ ]:
weights_filename = 'dqn_walls_weights.h5f'
checkpoint_weights_filename = 'dqn_' + "walls" + '_weights_{step}.h5f'
checkpoint_callback = ModelIntervalCheckpoint(checkpoint_weights_filename, interval=100000)


In [19]:
dqn.fit(env, nb_steps=1200000, callbacks=[checkpoint_callback], log_interval=100000, visualize=False)

# After training is done, we save the final weights one more time.
dqn.save_weights(weights_filename, overwrite=True)



Training for 1200000 steps ...
Interval 1 (0 steps performed)


/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/usr/local/lib/python3.8/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automa

    38/100000 [..............................] - ETA: 6:58 - reward: -0.0526   

/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


100000/100000 [==============================] - 578s 6ms/step - reward: -0.0411
4342 episodes - episode_reward: -0.947 [-1.000, 1.000] - loss: 0.004 - mae: 0.154 - mean_q: 0.198 - mean_eps: 0.932 - score: 0.051

Interval 2 (100000 steps performed)
100000/100000 [==============================] - 823s 8ms/step - reward: -0.0353
3804 episodes - episode_reward: -0.928 [-1.000, 2.000] - loss: 0.002 - mae: 0.396 - mean_q: 0.550 - mean_eps: 0.865 - score: 0.069

Interval 3 (200000 steps performed)
100000/100000 [==============================] - 920s 9ms/step - reward: -0.0315
3535 episodes - episode_reward: -0.891 [-1.000, 2.000] - loss: 0.003 - mae: 0.714 - mean_q: 0.987 - mean_eps: 0.775 - score: 0.105

Interval 4 (300000 steps performed)
100000/100000 [==============================] - 1042s 10ms/step - reward: -0.0243
3105 episodes - episode_reward: -0.781 [-1.000, 2.000] - loss: 0.007 - mae: 1.107 - mean_q: 1.546 - mean_eps: 0.685 - score: 0.199

Interval 5 (400000 steps performed)
10

In [20]:
# Load the weights
model.load_weights("dqn_walls_weights.h5f")


memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1, value_min=.1, value_test=.05,
                              nb_steps=100000)

processor = ImageProcessor()

# Initialize the DQNAgent with the new model and updated policy and compile it
dqn = DQNAgent(model=model, nb_actions=nb_actions, policy=policy, memory=memory,
               processor=processor, nb_steps_warmup=50000, gamma=.99, target_model_update=10000)
dqn.compile(Adam(learning_rate=.00025), metrics=['mae'])


In [21]:
env.sleep = 0.2

In [ ]:
dqn.test(env, nb_episodes=10, visualize=True)

Testing for 10 episodes ...


/usr/local/lib/python3.8/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initialization instead.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(
/usr/local/lib/python3.8/dist-packages/gym/utils/passive_env_checker.py:280: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


Episode 1: reward: 14.000, steps: 322
Episode 2: reward: 3.000, steps: 94
